In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from numpy.random import rand

In [2]:
#from sklearn import datasets
#iris = datasets.load_iris() 
#print(iris)
  
iris_df = pd.read_csv("C:/Users/ankit/Desktop/iris.data",header=None) # , columns=['Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width'] 
np.random.seed(42)

iris_df.columns = ['Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width', 'Target']

# iris_df['Target'].replace('Iris-setosa', [0], inplace=True) 
# iris_df['Target'].replace('Iris-vercicolor',[1], inplace=True) 
# iris_df['Target'].replace('Iris-virginica',[2], inplace=True)

iris_df['Target'] = np.where(iris_df['Target'] == 'Iris-setosa', 0.0, iris_df['Target'])
iris_df['Target'] = np.where(iris_df['Target'] == 'Iris-versicolor', 1.0, iris_df['Target'])
iris_df['Target'] = np.where(iris_df['Target'] == 'Iris-virginica', 2.0, iris_df['Target'])

#iris_df.Target[iris_df.Target == 'Iris-setosa'] = 0.0
#iris_df.Target[iris_df.Target == 'Iris-versicolor'] = 1.0
#iris_df.Target[iris_df.Target == 'Iris-virginica'] = 2.0

#iris_df = iris_df[['']] 
#iris_df['Target'] = iris.target 
#iris_df['Target'].value_counts()
#iris_df = iris_df[iris_df.Target != 2]
#iris_df['Target'].value_counts()
df = iris_df
df = np.array(df) 
np.random.shuffle( df)

eightypercent = int(0.8 * len(df))
twentypercent = int(0.2 * len(df))
_ ,features = df.shape
features = features - 1

df_train = df[:eightypercent,:features].reshape(eightypercent, features)
df_train_y = df[:eightypercent,-1].reshape(eightypercent, 1)

df_test = df[:twentypercent,:features].reshape(twentypercent,features)
df_test_y = df[:twentypercent,-1].reshape(twentypercent,1)

print("df_train.shape = ",df_train.shape,"df_train_y.shape = ",df_train_y.shape)
print("df_test.shape = ",df_test.shape,"df_test_y.shape = ",df_test_y.shape)



df_train.shape =  (120, 4) df_train_y.shape =  (120, 1)
df_test.shape =  (30, 4) df_test_y.shape =  (30, 1)


In [3]:
class LinearReg:

    def fit(self, X, y_class):
        # Least Squares
        N_instances, N_features = X.shape
        intercept = np.ones((X.shape[0], 1))
        X = np.hstack((intercept, X))
        #print(X)       
        self.weights = np.zeros(X.shape[1]).reshape(X.shape[1], 1) # 3 weights now not 2
        self.weights = np.dot(np.linalg.inv(np.dot(np.transpose(X), X)), np.dot(np.transpose(X), y_class))
        self.weights = self.weights.reshape(len(self.weights), 1)
        print("weights.shape = ",self.weights.shape)
        
        #print("self.weights", (self.weights).shape)
        #print("N_instances = ",N_instances,"N_features",N_features)
        
        return self.weights
        #print(o_class.shape)
            
    def prediction(self, X): #for training and testing data both we want to predict
        N_instances, N_features = X.shape
        intercept = np.ones((X.shape[0], 1))
        X = np.hstack((intercept, X))        
        
        #net = np.array(np.dot(X,self.weights)) #+ self.bias
        #np.array(net)
        #print((net))
        #print("net shape",net.shape)
        y = np.dot(X, self.weights)
        y = np.array(y)
        
        y = [1 if (i >= 0.5 and i < 1.5) else 0 if (i< 0.5 and i >=  -0.5) else 2 for i in y]
        y = np.array(y)
        
        return y

In [4]:
def accuracy(y_pred, y_actual):
    count = 0
    for i in range(len(y_actual)):
        if(y_pred[i] == y_actual[i]):
            count += 1
    accuracy = 100* ((count / len(y_actual)))
    print("accuracy = ", accuracy,"\n -----")
    return accuracy

In [5]:
## K- fold cross validation

k_two_to_ten = [i for i in range(2,11)]
print("k values from 2 to 10  = ",k_two_to_ten)
 
valid_k_values = list(filter(lambda k: len(df)%k == 0, k_two_to_ten)) 
print("valid_k_values divisible by len(dataset)= ",np.array(valid_k_values))

valid_k_values_list = []
for i in range(len(valid_k_values)):
    valid_k_values_list.append(int(valid_k_values[i]))  


valid_k_values = np.array(valid_k_values_list)

train_percent = ( ((valid_k_values - 1)/valid_k_values) * len(df))
test_percent = ( (1/valid_k_values) * len(df))
train_percent = train_percent.astype('int32')
test_percent = test_percent.astype('int32')
print("train_percent = ",train_percent)
print("test_percent = ",test_percent)

_ ,features = df.shape
features = features - 1

accuracies = []

for i in valid_k_values:
    
    print("k =",i,"\n")
    train_percent = ( ((i - 1)/i) * len(df))
    test_percent = ( (1/i) * len(df))
    train_percent = train_percent.astype('int32')
    test_percent = test_percent.astype('int32')

    df_train = df[:train_percent,:features].reshape(train_percent, features)
    df_train_y = df[:train_percent,-1].reshape(train_percent, 1)

    df_test = df[:test_percent,:features].reshape(test_percent,features)
    df_test_y = df[:test_percent,-1].reshape(test_percent,1)

    print("df_train.shape = ",df_train.shape,"df_train_y.shape = ",df_train_y.shape)
    print("df_test.shape = ",df_test.shape,"df_test_y.shape = ",df_test_y.shape)

    X_LinearReg =  LinearReg() 

    #print("X_train =",X_train)
    X_LinearReg.fit(df_train.astype(np.float64), df_train_y.astype(np.float64))

    #print("X_train =",X_train)
    PRED_CLASS_Test = X_LinearReg.prediction(df_test)
    #print(PRED_CLASS_Test)
    print(PRED_CLASS_Test.reshape(test_percent,1).shape)
    #print("For Test set:\n")

    acc = accuracy(PRED_CLASS_Test, df_test_y)

    
    accuracies.append(acc)

max_accuracy = max(accuracies)
index_max_accuracy = accuracies.index(max(accuracies))
print("max(accuracies) = ",max_accuracy,"index of max accuracy = " ,index_max_accuracy)
print("max accuracy's corresponding k-value = ", valid_k_values[index_max_accuracy])

k values from 2 to 10  =  [2, 3, 4, 5, 6, 7, 8, 9, 10]
valid_k_values divisible by len(dataset)=  [ 2  3  5  6 10]
train_percent =  [ 75 100 120 125 135]
test_percent =  [75 50 30 25 15]
k = 2 

df_train.shape =  (75, 4) df_train_y.shape =  (75, 1)
df_test.shape =  (75, 4) df_test_y.shape =  (75, 1)
weights.shape =  (5, 1)
(75, 1)
accuracy =  98.66666666666667 
 -----
k = 3 

df_train.shape =  (100, 4) df_train_y.shape =  (100, 1)
df_test.shape =  (50, 4) df_test_y.shape =  (50, 1)
weights.shape =  (5, 1)
(50, 1)
accuracy =  98.0 
 -----
k = 5 

df_train.shape =  (120, 4) df_train_y.shape =  (120, 1)
df_test.shape =  (30, 4) df_test_y.shape =  (30, 1)
weights.shape =  (5, 1)
(30, 1)
accuracy =  100.0 
 -----
k = 6 

df_train.shape =  (125, 4) df_train_y.shape =  (125, 1)
df_test.shape =  (25, 4) df_test_y.shape =  (25, 1)
weights.shape =  (5, 1)
(25, 1)
accuracy =  100.0 
 -----
k = 10 

df_train.shape =  (135, 4) df_train_y.shape =  (135, 1)
df_test.shape =  (15, 4) df_test_y.shape = 

## GRADIENT DESCENT

In [6]:
class LinearReg_Gradient_Descent:
    def __init__(self, LearningRate = 0.01, iterations = 10000):
        self.eta_lr = LearningRate
        self.total_iterations = iterations
        
        self.weights = weights
        #self.bias = bias
    
    
    def fit(self,X,y_class_groundtruth): #only for training data
        # Gradient Descent
        N_instances, N_features = X.shape

        
        intercept = np.ones((X.shape[0], 1))
        X = np.hstack((intercept, X))
        #print(X)       
        self.weights = np.zeros(X.shape[1]).reshape(X.shape[1], 1) # 3 weights now not 2
        #self.weights = np.dot(np.linalg.inv(np.dot(np.transpose(X), X)), np.dot(np.transpose(X), y_class))
        self.weights = self.weights.reshape(len(self.weights), 1)
        print("weights.shape = ",self.weights.shape)        
        
        y_class_onehotencode = np.array(pd.get_dummies(y_class_groundtruth['Target']))
        
        for no_of_iterations_to_converge in range(self.total_iterations):
            
            net = np.dot(X,self.weights) #+ self.bias
            net = net.astype('float64')
            o_class = (self.softmax(net)) 
            
            #ONES = np.ones((1,N_instances))
            #gradient_errorWRTbias = (1/N_instances)*np.dot(ONES,(o_class - y_class_onehotencode))
            gradient_errorWRTweights = (1/N_instances)*np.dot(X.T,(o_class - y_class_onehotencode))
            #self.bias = self.bias - self.eta_lr*gradient_errorWRTbias
            self.weights = self.weights - self.eta_lr*gradient_errorWRTweights
        
    def prediction(self, X): #for testing data both, we want to predict
        N_instances, N_features = X.shape
        intercept = np.ones((X.shape[0], 1))
        X = np.hstack((intercept, X))
        
        net = np.dot(X,self.weights) #+ self.bias
        net = net.astype('float64')
        y_softmax = self.softmax(net) 
        
        #print("y_softmax = ", y_softmax)
        #print("y_softmax max = ", max(y_softmax))
        #print("y_softmax min = ", min(y_softmax))
        
        o_class = np.argmax(y_softmax, axis = 1).reshape(N_instances,1)
    
        return o_class
                    
    def softmax(self,net):
        b = net.max()
        y = np.exp(net - b)
        return y / y.sum()
        #return np.exp(net)/(np.exp(net).sum(axis=0))


In [7]:
def accuracy_LinearReg_Gradient_Descent(y_class_true,y_class_pred,N_instances):
    count_accurate = 0
    y_class_true = np.array(y_class_true).reshape(N_instances,1)
    y_class_pred = np.array(y_class_pred).reshape(N_instances,1)
    
   # print(y_class_true[0])
    for i in range(N_instances):
        if(y_class_true[i] == y_class_pred[i]):
            count_accurate = count_accurate + 1
    
    accuracy = (count_accurate/len(y_class_true))*100
    return accuracy


In [8]:
## K- fold cross validation

k_two_to_ten = [i for i in range(2,11)]
print("k values from 2 to 10  = ",k_two_to_ten)
 
valid_k_values = list(filter(lambda k: len(df)%k == 0, k_two_to_ten)) 
print("valid_k_values divisible by len(dataset)= ",np.array(valid_k_values))

valid_k_values_list = []
for i in range(len(valid_k_values)):
    valid_k_values_list.append(int(valid_k_values[i]))  


valid_k_values = np.array(valid_k_values_list)

train_percent = ( ((valid_k_values - 1)/valid_k_values) * len(df))
test_percent = ( (1/valid_k_values) * len(df))
train_percent = train_percent.astype('int32')
test_percent = test_percent.astype('int32')
print("train_percent = ",train_percent)
print("test_percent = ",test_percent)

print("\n\n---------------\n\n")

_ ,features = df.shape
features = features - 1

accuracies = []

for i in valid_k_values:
    print("k =",i,"\n")
    train_percent = ( ((i - 1)/i) * len(df))
    test_percent = ( (1/i) * len(df))
    train_percent = train_percent.astype('int32')
    test_percent = test_percent.astype('int32')

    df_train = df[:train_percent,:features].reshape(train_percent, features)
    df_train_y = df[:train_percent,-1].reshape(train_percent, 1)

    df_test = df[:test_percent,:features].reshape(test_percent,features)
    df_test_y = df[:test_percent,-1].reshape(test_percent,1)

    print("df_train.shape = ",df_train.shape,"df_train_y.shape = ",df_train_y.shape)
    print("df_test.shape = ",df_test.shape,"df_test_y.shape = ",df_test_y.shape)
    
    classes = np.unique(df_train_y)
    N_classes = len(classes)
    print("N_classes =",N_classes)
    N_instances_train,N_features_train = df_train.shape #N,D
    N_instances_test,N_features_test = df_test.shape # N,D ---- we know, N_features_train = N_features_test
    print("N_instances_train",N_instances_train,"N_features_train",N_features_train,"N_instances_test",N_instances_test,"N_features_test",N_features_test)


    weights = np.zeros((N_features_train + 1, N_classes))
    
    df_train_y_pd = pd.DataFrame(df_train_y)
    df_train_pd = pd.DataFrame(df_train)


    df_train_y_pd.columns = ['Target']

    df_test_y_pd = pd.DataFrame(df_test_y)
    df_test_pd = pd.DataFrame(df_test)


    df_test_y_pd.columns = ['Target']
    

    X_LinearReg_Gradient_Descent =  LinearReg_Gradient_Descent() #Default Learning Rate

    X_LinearReg_Gradient_Descent.fit(df_train_pd,df_train_y_pd)
    y_class_pred_test = X_LinearReg_Gradient_Descent.prediction(df_test_pd)
    

    
    
    acc = accuracy_LinearReg_Gradient_Descent(df_test_y,y_class_pred_test,len(df_test_y))
    Accuracy_Testing = print("Accuracy of model on testing data",acc,"%\n\n---------\n")
    
    #X_LinearReg =  LinearReg() 

    
    #X_LinearReg.fit(df_train.astype(np.float64), df_train_y.astype(np.float64))

    
    #PRED_CLASS_Test = X_LinearReg.prediction(df_test)
    
    #print(PRED_CLASS_Test.reshape(test_percent,1).shape)
    
    #acc = accuracy(PRED_CLASS_Test, df_test_y)

    
    accuracies.append(acc)

max_accuracy = max(accuracies)
index_max_accuracy = accuracies.index(max(accuracies))
print("max(accuracies) = ",max_accuracy,"index of max accuracy = " ,index_max_accuracy)
print("max accuracy's corresponding k-value = ", valid_k_values[index_max_accuracy])


k values from 2 to 10  =  [2, 3, 4, 5, 6, 7, 8, 9, 10]
valid_k_values divisible by len(dataset)=  [ 2  3  5  6 10]
train_percent =  [ 75 100 120 125 135]
test_percent =  [75 50 30 25 15]


---------------


k = 2 

df_train.shape =  (75, 4) df_train_y.shape =  (75, 1)
df_test.shape =  (75, 4) df_test_y.shape =  (75, 1)
N_classes = 3
N_instances_train 75 N_features_train 4 N_instances_test 75 N_features_test 4
weights.shape =  (5, 1)
Accuracy of model on testing data 68.0 %

---------

k = 3 

df_train.shape =  (100, 4) df_train_y.shape =  (100, 1)
df_test.shape =  (50, 4) df_test_y.shape =  (50, 1)
N_classes = 3
N_instances_train 100 N_features_train 4 N_instances_test 50 N_features_test 4
weights.shape =  (5, 1)
Accuracy of model on testing data 70.0 %

---------

k = 5 

df_train.shape =  (120, 4) df_train_y.shape =  (120, 1)
df_test.shape =  (30, 4) df_test_y.shape =  (30, 1)
N_classes = 3
N_instances_train 120 N_features_train 4 N_instances_test 30 N_features_test 4
weights.shape =